In [2]:
 %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [289]:
data = pd.read_csv('raw city data/tucson_az.csv')
# data.head()
data.shape

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1395365, 22)

In [290]:
data.head()

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,...,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Tucson Police Department Neighborhood Associations Shapes - t3ee-ue34,Tucson Police Department Divisions Shapes - 8bu4-xug5
0,826668730,1711100063,11/09/2017 02:56:00 AM,3212,TRAFFIC ACCIDENT/PRP DMG/HIT-AND-RUN/MOTOR VEH...,NaN,S CAMPBELL AV & E DREXEL RD,NaN,TUCSON,AZ,...,32.148676,-110.943456,11/14/2017 01:49:08 PM,11/24/2017 10:16:30 PM,POINT (-110.94345620176111 32.14867586273264),2,Thursday,Traffic,412306.0,569574.0
1,826668732,1711100065,11/09/2017 03:13:00 AM,2702,ARREST FOR OTHER JURISDICTION/MISD CRIMINL WAR...,NaN,500 Block W AJO WAY,NaN,TUCSON,AZ,...,32.177864,-110.977518,11/14/2017 01:49:08 PM,11/24/2017 10:16:30 PM,POINT (-110.97751830144006 32.17786425688853),3,Thursday,Community Policing,NaN,569570.0
2,758053092,1604250572,04/25/2016 06:09:00 PM,Other,Contact the Tucson Police Department if you ha...,NaN,3800 Block N FLOWING WELLS RD,NaN,TUCSON,AZ,...,32.275835,-110.994943,04/27/2016 08:37:32 AM,05/25/2016 06:51:07 PM,POINT (-110.994942815655 32.2758349176723),18,Monday,Other,412227.0,569577.0
3,826668737,1711100080,11/09/2017 03:37:00 AM,0912,ASSAULT/NO INJURY - DOMESTIC VIOLENCE<br/><br/...,NaN,700 Block E UNIVERSITY BLVD,NaN,TUCSON,AZ,...,32.231624,-110.960955,11/14/2017 01:49:08 PM,11/24/2017 10:16:30 PM,POINT (-110.96095460720035 32.231624299257405),3,Thursday,Assault,412292.0,569582.0
4,826668751,1711100120,11/09/2017 04:00:00 AM,0604,LARCENY/FROM MOTOR VEHICLE<br/><br/>Contact th...,NaN,3200 Block E HAWTHORNE ST,NaN,TUCSON,AZ,...,32.233308,-110.925059,11/14/2017 01:49:08 PM,11/24/2017 10:16:32 PM,POINT (-110.9250593571503 32.23330760135101),4,Thursday,Theft from Vehicle,412246.0,569563.0


In [291]:
# data['Location1'] = ""
# data.Location1 = data["Latitude"].map(str) + data["Longitude"].map(str)
tb1 = data[['location']].copy()
tb1.head()
tb1.shape

(1395365, 1)

In [314]:
# data['Location'].value_counts()

# shows how many crimes occurred in each zipcode
new_table = data['location'].value_counts().rename_axis('Location').reset_index(name='total_crimes')
# new_table['Location']=new_table['Location'].str.split('\n',expand=True)[1]
# new_table.head()
# new_table['zipcode']=new_table['Location'].str.split('CITY',expand=True)[1]
new_table['lat1']=new_table['Location'].str.split(' ',expand=True)[1]
new_table['long1']=new_table['Location'].str.split(' ',expand=True)[2]
new_table['long']=new_table['lat1'].str.split('(',expand=True)[1]
new_table['lat']=new_table['long1'].str.split(')',expand=True)[0]

new_table = new_table.drop(['lat1','long1'], axis = 1)

new_table['lat']=pd.to_numeric(new_table['lat'])
new_table['long']=pd.to_numeric(new_table['long'])

new_table.head()


,Location,total_crimes,long,lat
0,POINT (-110.926479 32.221743),24064,-110.926479,32.221743
1,POINT (-110.968719 32.18603),2788,-110.968719,32.186030
2,POINT (-110.970583541983 32.2177385304574),2407,-110.970584,32.217739
3,POINT (-110.908578 32.249682),2049,-110.908578,32.249682
4,POINT (-110.993822 32.178578),1874,-110.993822,32.178578


In [315]:
data.shape

(1395365, 22)

In [316]:
new_table.shape

(138402, 4)

In [317]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

for index, row in new_table.iterrows():
    lat = row['lat']
    long = row['long']
    zipcode = search.by_coordinates(lat, long, returns = 1) 
    if len(zipcode) > 0:
        blah = zipcode[0].zipcode
    else:
        blah = 0
    new_table.at[index, 'zipcode'] = blah
    if index%1000 == 0:
        print(str(index) + " still going")

0 still going
1000 still going
2000 still going
3000 still going
4000 still going
5000 still going
6000 still going
7000 still going
8000 still going
9000 still going
10000 still going
11000 still going
12000 still going
13000 still going
14000 still going
15000 still going
16000 still going
17000 still going
18000 still going
19000 still going
20000 still going
21000 still going
22000 still going
23000 still going
24000 still going
25000 still going
26000 still going
27000 still going
28000 still going
29000 still going
30000 still going
31000 still going
32000 still going
33000 still going
34000 still going
35000 still going
36000 still going
37000 still going
38000 still going
39000 still going
40000 still going
41000 still going
42000 still going
43000 still going
44000 still going
45000 still going
46000 still going
47000 still going
48000 still going
49000 still going
50000 still going
51000 still going
52000 still going
53000 still going
54000 still going
55000 still going
56000

In [320]:
# tb1.at[179, 'zipcode'] = 64114
# table2 = tb1.drop(['Latitude','Longitude'], axis=1)
# d = {'total_crimes': 'sum'}
# table2 = new_table.groupby(new_table['zipcode']).aggregate(d).reset_index()
# table2 = table2.drop([0])
# table2 = table2[table2.zipcode < 64000]
# table2 = table2.drop([98,99])

# table2 = table2.sort_values(by=['zipcode'])
# table2 = table2.reset_index(drop=True)
# table2.zipcode = pd.to_numeric(table2.zipcode)

# table2 = tb1['zipcode'].value_counts().rename_axis('zipcode').reset_index(name='total_crimes')

table2.head(200)

,zipcode,total_crimes
1,19146,1
2,27608,1
3,32504,1
4,33012,1
5,34235,1
6,37042,1
7,37149,2
8,48505,1
9,56001,1
10,60626,1


In [321]:
table2.zipcode = pd.to_numeric(table2.zipcode)
table2.zipcode.dtype

dtype('int64')

In [322]:
census = pd.read_csv('census_data.csv')
census.head()
# new_table.head()

# new_table.zipcode.dtype

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,601,18450,36.6,12041.0,7380.0,10816,58.623306
1,602,41302,38.6,15663.0,8463.0,22409,54.256452
2,603,53683,38.9,15485.0,9176.0,26220,48.842278
3,606,6591,37.3,15019.0,6383.0,3721,56.455773
4,610,28963,39.2,16707.0,7892.0,14569,50.302110


In [323]:
tb2 = table2.merge(census, left_on='zipcode', right_on='Zipcode', how='left')
tb2.head(200)

,zipcode,total_crimes,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,19146,1,19146,34837,33.2,42068.0,32882.0,8197,23.529581
1,27608,1,27608,10644,39.2,81351.0,57716.0,721,6.773769
2,32504,1,32504,21859,41.3,44837.0,26934.0,2604,11.912713
3,33012,1,33012,73805,44.6,32273.0,15832.0,15920,21.570354
4,34235,1,34235,13462,53.6,48750.0,33455.0,1886,14.009805
5,37042,1,37042,68419,27.3,46477.0,19578.0,12577,18.382321
6,37149,2,37149,2331,45.6,53828.0,24847.0,139,5.963106
7,48505,1,48505,23343,34.1,21559.0,11174.0,10185,43.631924
8,56001,1,56001,46689,27.6,44926.0,24452.0,10445,22.371437
9,60626,1,60626,51061,33.0,38734.0,24054.0,13330,26.106030


In [325]:

# tb2 = tb2.dropna()
# d = {'total_crimes': 'sum'}
# table2 = tb2.groupby(tb2['zipcode']).aggregate(d).reset_index()
# table2.head()
# tb2 = table2.merge(census, left_on='zipcode', right_on='Zipcode', how='left')
tb2.head(200)

,zipcode,total_crimes,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,19146,1,19146,34837,33.2,42068.0,32882.0,8197,23.529581
1,27608,1,27608,10644,39.2,81351.0,57716.0,721,6.773769
2,32504,1,32504,21859,41.3,44837.0,26934.0,2604,11.912713
3,33012,1,33012,73805,44.6,32273.0,15832.0,15920,21.570354
4,34235,1,34235,13462,53.6,48750.0,33455.0,1886,14.009805
5,37042,1,37042,68419,27.3,46477.0,19578.0,12577,18.382321
6,37149,2,37149,2331,45.6,53828.0,24847.0,139,5.963106
7,48505,1,48505,23343,34.1,21559.0,11174.0,10185,43.631924
8,56001,1,56001,46689,27.6,44926.0,24452.0,10445,22.371437
9,60626,1,60626,51061,33.0,38734.0,24054.0,13330,26.106030


In [326]:
tb2['crime rate'] = tb2.total_crimes / tb2.Population
tb2.sort_values(by=['crime rate'], ascending=False)
tb2 = tb2.reset_index(drop = True)

In [327]:
tb2.drop(['Zipcode'], axis=1).head()

,zipcode,total_crimes,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,crime rate
0,19146,1,34837,33.2,42068.0,32882.0,8197,23.529581,0.000029
1,27608,1,10644,39.2,81351.0,57716.0,721,6.773769,0.000094
2,32504,1,21859,41.3,44837.0,26934.0,2604,11.912713,0.000046
3,33012,1,73805,44.6,32273.0,15832.0,15920,21.570354,0.000014
4,34235,1,13462,53.6,48750.0,33455.0,1886,14.009805,0.000074


In [330]:
tb2 = tb2.sort_values(by=['crime rate'], ascending=False)
tb2 = tb2.reset_index(drop=True)
tb2 = tb2.drop([0,1,2,3,4,5,6,7,8,9,10,11])
tb2 = tb2.reset_index(drop=True)
tb2.head(100)

,zipcode,total_crimes,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,crime rate
0,85746,29633,85746,42704,31.8,41427.0,16176.0,10686,25.023417,0.693916
1,85706,38102,85706,56989,27.5,29814.0,12164.0,20916,36.701820,0.668585
2,85756,8557,85756,35522,31.8,46672.0,16574.0,5229,14.720455,0.240893
3,85747,4670,85747,23490,34.5,81474.0,31039.0,1199,5.104300,0.198808
4,85748,2971,85748,18275,45.4,69871.0,34353.0,1254,6.861833,0.162572
5,85321,433,85321,4727,45.2,28053.0,17453.0,1782,37.698329,0.091601
6,85634,546,85634,5982,29.4,27854.0,10733.0,2290,38.281511,0.091274
7,65555,79,65555,1077,52.1,38125.0,18221.0,185,17.177344,0.073352
8,85741,1913,85741,35191,34.7,51871.0,23064.0,4306,12.236083,0.054360
9,85745,1853,85745,37654,36.8,49380.0,27133.0,7916,21.022999,0.049211


In [331]:
tb2.to_csv('tucson_census.csv', encoding='utf-8', index=False)